In [1]:
import aocd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from itertools import permutations
from copy import copy
data = list([int(d) for d in aocd.get_data().split(",")])
print(data)

[1, 330, 331, 332, 109, 3546, 1101, 0, 1182, 15, 1101, 1481, 0, 24, 1001, 0, 0, 570, 1006, 570, 36, 102, 1, 571, 0, 1001, 570, -1, 570, 1001, 24, 1, 24, 1105, 1, 18, 1008, 571, 0, 571, 1001, 15, 1, 15, 1008, 15, 1481, 570, 1006, 570, 14, 21102, 58, 1, 0, 1106, 0, 786, 1006, 332, 62, 99, 21101, 0, 333, 1, 21101, 0, 73, 0, 1106, 0, 579, 1101, 0, 0, 572, 1101, 0, 0, 573, 3, 574, 101, 1, 573, 573, 1007, 574, 65, 570, 1005, 570, 151, 107, 67, 574, 570, 1005, 570, 151, 1001, 574, -64, 574, 1002, 574, -1, 574, 1001, 572, 1, 572, 1007, 572, 11, 570, 1006, 570, 165, 101, 1182, 572, 127, 1002, 574, 1, 0, 3, 574, 101, 1, 573, 573, 1008, 574, 10, 570, 1005, 570, 189, 1008, 574, 44, 570, 1006, 570, 158, 1105, 1, 81, 21102, 1, 340, 1, 1106, 0, 177, 21102, 1, 477, 1, 1106, 0, 177, 21101, 0, 514, 1, 21102, 1, 176, 0, 1105, 1, 579, 99, 21102, 1, 184, 0, 1106, 0, 579, 4, 574, 104, 10, 99, 1007, 573, 22, 570, 1006, 570, 165, 102, 1, 572, 1182, 21102, 375, 1, 1, 21101, 211, 0, 0, 1106, 0, 579, 21101, 1182

In [2]:
IN_SIZE = {1: 4, 2: 4, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4, 8: 4, 9: 2, 99: 1}
IN_NAME =  {1: "add", 2: "mul", 3: "rd", 4: "prnt", 5: "jnz", 6: "jz", 
            7: "lt", 8: "eq", 9: "bas", 99: "ret"}
EXT_MEM = 10000

class Process():  # wrapper for generator
    def __init__(self, data, ptr=0, dbg=False): 
        self.d = data[:]+[0]*EXT_MEM  # copy + extend memory
        self.done = False
        self.base = 0
        self.ptr = ptr
    
    def parse_ins(self, ptr, dbg=False):
        param = [0, 0, 0]
        ins = self.d[ptr]%100
        modes = [self.d[ptr]//10**e%10 for e in range(2,5)]
        for i, mode in enumerate(modes):
            size = IN_SIZE[ins]-1
            if i < size:
                p = ptr+1+i
                if mode == 0:  param[i] = self.d[p]          # position
                if mode == 1:  param[i] = p                  # intermediate
                if mode == 2:  param[i] = self.base+self.d[p]# relative
        if dbg: print(ptr, IN_NAME[ins], param[:size], 
                      self.d[ptr:ptr+4], sep = "\t")# debug print
        return [ins] + param

    def process(self, inp, dbg=False): 
        out = []; ptr = self.ptr; d = self.d  # initializations
        parse = lambda i: [i%100] + [i//10**e%10 for e in range(2,5)]
        while ptr < len(d):                                     # stop on EOF
            ins, p1, p2, p3 = self.parse_ins(ptr, dbg=dbg)
            # if dbg:print(ptr, d)                              # debug print
            if   ins == 1: d[p3] = d[p1] + d[p2]                # add
            elif ins == 2: d[p3] = d[p1] * d[p2]                # mul
            elif ins == 3:                                      # read
                if not inp: self.ptr = ptr; return out;         # wait/flush
                d[p1] = inp.pop(0)                              # read
            elif ins == 4: out.append(d[p1])                    # print
            elif ins == 5: ptr = d[p2]-3 if     d[p1] else ptr  # jnz
            elif ins == 6: ptr = d[p2]-3 if not d[p1] else ptr  # jz
            elif ins == 7: d[p3] = int(d[p1] < d[p2])           # lt
            elif ins == 8: d[p3] = int(d[p1] == d[p2])          # eq
            elif ins == 9: self.base += d[p1]                   # base
            elif ins ==99: self.done=True; return out           # ret
            else: print(f"invalid opcode {ins} @ {ptr}")        # err
            ptr += IN_SIZE[ins] # jmp is compensated with -3    # move ptr

def chain_process(prog, inl, out=[0], dbg=False):
    proc = [Process(prog, dbg=dbg) for inp in inl]
    for epoch in range(100000):
        for i, inp in enumerate(inl):
            out = proc[i].process(([inp] if epoch == 0 else [])+out)
            if proc[i].done and i == len(inl)-1:  # if the last is done, get out
                return out[0]

# tests in other files

In [49]:
parse =   lambda text:  np.array([list(c) for c in text.strip().split("\n")])
arr2str = lambda lines: "\n".join(["".join(line) for line in lines])
calc =    lambda field: sum([p[0]*p[1] for p in np.array(np.where(seto(field)=="O")).T]) # sum of O-scores
il2str = lambda il: "".join([chr(i) for i in il])

def seto(field, dbg=False):
    for pos, val in np.ndenumerate(field):
        try:
            cmp = np.array([field[tuple(np.array(pos)+p)] 
                             for p in [[0,0], [0,1], [1,0], [-1,0], [0,-1]]])
            if all(cmp=="#"): field[tuple(pos)] = "O"
        except IndexError: pass
    if dbg: print(arr2str(field))
    return field

assert calc(parse("""
..#..........
..#..........
##O####...###
#.#...#...#.#
##O###O###O##
..#...#...#..
..#####...^..""")) == 76

solution = calc(parse(il2str(Process(data).process(inp=[1]))))
aocd.submit(solution, day=17)

answer a: 5972
submitting for part b (part a is already completed)
posting 0 to https://adventofcode.com/2019/day/17/answer (part b) token=...2749


You don't seem to be solving the right level.  Did you already complete it? [Return to Day 17]


<Response [200]>

In [52]:
def frame(field):  # draw a "." frame around view
    nf = np.full((field.shape[0]+2, field.shape[1]+2), ".")
    nf[1:-1, 1:-1] = field
    return nf

c2t = lambda pos: int(pos[0])+int(pos[1])*1j             # complex 2 tuple
t2c = lambda pos: tuple([int(pos.real), int(pos.imag)])  # tuple 2 complex
a = 2+3j; assert a == c2t(t2c(a))

def findpath(field, currdir = -1):
    field = frame(field)
    pos = c2t(np.where(field == "^"))
    movement = []
    for _ in range(100):
        for dire in [1,3]:
            ndir = currdir * (1j**dire)
            npos = pos + ndir
            if field[t2c(npos)] == '#':
                movement+=["L"] if dire==1 else ["R"]
            else:
                continue
            for dist in range(13):
                if field[t2c(pos+ndir*(dist+1))] != "#":
                    break
            movement+=[str(dist)]
            pos += ndir * dist
            currdir = ndir
    return movement

assert findpath(parse("""
#######...#####.
#.....#...#...#.
#.....#...#...#.
......#...#...#.
......#...###.#.
......#.....#.#.
^########...#.#.
......#.#...#.#.
......#########.
........#...#...
....#########...
....#...#.......
....#...#.......
....#...#.......
....#####.......
................""")) == "R,8,R,8,R,4,R,4,R,8,L,6,L,2,R,4,R,4,R,8,R,8,R,8,L,6,L,2".split(",")

line = "A,A,B,C,B,C,B,A,C,A"
A = "R,8,L,12,R,8"
B = "L,10,L,10,R,8"
C = "L,12,L,12,L,10,R,10"

inp = "\n".join([line, A, B, C, "n\n"])
inp = [ord(c) for c in inp]
data[0] = 2
rd = Process(data).process(inp=inp)
print(il2str(rd))
aocd.submit(rd[-1], day=17)

answer a: 5972
submitting for part b (part a is already completed)
posting 933214 to https://adventofcode.com/2019/day/17/answer (part b) token=...2749


......#############...........................#############
......#...........#...........................#...........#
......#...........#...........................#...........#
......#...........#...........................#...........#
......#...........#...........................#...........#
......#...........#...........................#...........#
......#...........#.#########.........###########.........#
......#...........#.#.......#.........#.......#.#.........#
......#...........#############.......#.......#.#.........#
......#.............#.......#.#.......#.......#.#.........#
......#.............#.......#.#.....###########.#.........#
......#.............#.......#.#.....#.#.........#.........#
......###########...#.......#.#.....#.#.........#.#########
................#...#.......#.#.....#.#.........#.#........
................#...#.......#############.......#.#........
................#...#.........#.....#.#.#.......#.#........
................#...#.........#########.

<Response [200]>